# Interpretable Regression

In [ ]:
# install interpret if not already installed
try:
    import interpret
except ModuleNotFoundError:
    !pip install --quiet interpret pandas scikit-learn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from interpret import show
from interpret.perf import RegressionPerf

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

X, y = load_diabetes(return_X_y=True, as_frame=True)

seed = 42
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

<h2>Explore the dataset</h2>

In [ ]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

<h2>Train the Explainable Boosting Machine (EBM)</h2>

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

<h2>EBMs are glassbox models, so we can edit them</h2>

In [ ]:
# post-process monotonize the age feature
ebm.monotonize("age", increasing=True)

<h2>Global Explanations: What the model learned overall</h2>

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<h2>Local Explanations: How an individual prediction was made</h2>

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local, 0)

<h2>Evaluate EBM performance</h2>

In [ ]:
ebm_perf = RegressionPerf(ebm).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<h2>Let's test out a few other Explainable Models</h2>

In [ ]:
from interpret.glassbox import LinearRegression, RegressionTree

lr = LinearRegression()
lr.fit(X_train, y_train)

rt = RegressionTree(random_state=seed)
rt.fit(X_train, y_train)

<h2>Compare performance using the Dashboard</h2>

In [ ]:
lr_perf = RegressionPerf(lr).explain_perf(X_test, y_test, name='Linear Regression')
show(lr_perf)

In [ ]:
rt_perf = RegressionPerf(rt).explain_perf(X_test, y_test, name='Regression Tree')
show(rt_perf)

<h2>Glassbox: All of our models have global and local explanations</h2>

In [ ]:
lr_global = lr.explain_global(name='Linear Regression')
show(lr_global)

In [ ]:
rt_global = rt.explain_global(name='Regression Tree')
show(rt_global)

<h2>Dashboard: look at everything at once</h2>

In [ ]:
# Do everything in one shot with the InterpretML Dashboard by passing a list into show

show([marginal, lr_global, lr_perf, rt_global, rt_perf, ebm_global, ebm_perf])